# Chapter 8. Object References, Mutability, and Recycling

파이썬 변수를 은유적으로 표현한다. 변수는 이름표지 상자 자체가 아니다. 참조 변수를 알고 있다면 다른 사람에게 별명(alias)문제를 설명할 때 이 비유가 도움이 될 것이다.(?)  
**변수가 데이터를 저장하는 일종의 상자와 같다고 배운 것은 잊어버리자!**

## 8.1 변수는 상자가 아니다
상자로서의 변수 라는 개념이 실제로 객체지향 언어에서 참조변수를 이해하는 데 방해가 된다고 강조했다. 파이썬 변수는 자바에서의 참조 변수와 같으므로 변수는 객체에 붙은 레이블이라고 생각하는 것이 좋다.  
(여기서는 포스트잇이라고 표현하네요)

In [1]:
a = [1, 2, 3]
b = a
a.append(4)
print(b)

[1, 2, 3, 4]


In [3]:
class Gizmo:
    def __init__(self):
        print('Gizmo id: %d' % id(self))

x = Gizmo() # = 는 할당 연산자, 객체를 생성하고 그 객체를 가리키는 변수를 만든다.
y = Gizmo() * 10

Gizmo id: 4400789104
Gizmo id: 4400791360


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [4]:
dir()

['Gizmo',
 'In',
 'Out',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__vsc_ipynb_file__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'b',
 'exit',
 'get_ipython',
 'os',
 'quit',
 'sys',
 'x']

In [40]:
import weakref
a_set = {0, 1}
wref = weakref.ref(a_set)
wref

<weakref at 0x107586c70; to 'set' at 0x1074a8e40>

In [47]:
wref()

{0, 1}

In [48]:
_   # 어이없는 변수가 나옵니다.

{0, 1}

In [56]:
a = {2, 3, 4}
print(wref())
print(_)
print(wref() is None)
print(wref() is None)   # 여기선 False가 나오긴 하지만 인터프리터에서 상에서는 True가 나옵니다.


{0, 1}
False
False
False


interpreter
```
>>> import weakref
>>> a_set = {0,1}
>>> wref = weakref.ref(a_set)
>>> wref
<weakref at 0x102f4a220; to 'set' at 0x1030bd040>
>>> wref()
{0, 1}
>>> a_set = {2,3,4}
>>> wref()
{0, 1}
>>> wref() is None
False
>>> _
False
>>> type(__
... )
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name '__' is not defined
>>> type(_)
<class 'bool'>
>>> wref() is None
True
>>> wref() is None
True
>>> wref() is None
True
>>> wref() is None
True
>>> 
>>> wref() is None
True
```

# Note

## Call by sharing

In [5]:
# immutable
def foo(x):
    # x와 y는 서로 다른 변수이다.
    print('x is %d' % id(x))
    x = 3
    print('Changed local x to %d' % id(x))

y = 5
print('y is %d' % id(y))
foo(y)

y is 4339803936
x is 4339803936
Changed local x to 4339803872


In [8]:
# mutable
def foo(x):
    # x와 y는 동일한 객체를 참조하고 있다.
    x[0] = 3
    print('x is %d' % id(x))

y = [5]
foo(y)
print(y)
print('y is %d' % id(y))

x is 4417464256
[3]
y is 4417464256


In [13]:
x = 1
y = [x, 2, 3]
y[0] = 4

print(x, id(x))
print(y, id(y))

1 4339803808
[4, 2, 3] 4417416640


## mutable한 객체를 기본매개변수로 사용하는 것은 위험하다
기본 매개변수는 함수의 `__defaults__` 속성에 저장된다. 해당 속성은 함수가 처음 정의 될 때 초기화 된 이후 지속적으로 사용된다.

In [14]:
def foo(x=[]):
    x.append(1)
    print(foo.__defaults__)
    print(x)

In [38]:
foo.__defaults__ = set("a")

TypeError: __defaults__ must be set to a tuple object

In [27]:
foo()
foo()
foo()
foo()

([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],)
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],)
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],)
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],)
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [28]:
foo(y)

([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],)
[4, 2, 3, 1]


In [39]:
class Bus:
    def __init__(self, passengers=None)-> None:
        if passengers:
            self.passengers = passengers
        else:
            self.passengers = []

    def drop(self, name):
        self.passengers.remove(name)


team = ['a', 'b', 'c']
bus = Bus(team)
bus.drop('a')
print(bus.passengers, team)


['b', 'c'] ['b', 'c']
